[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//saeidtafazzol/Indirect-Method-for-Low-Thrust-Trajectory-Design-in-Python-/blob/main/code.ipynb)


# Space Craft Dynamics

In this section, we will talk about the dynamics of the spacecraft assuming that it is only affected by the gravity field of **one** central body.

Let $\boldsymbol{r} = [x,y,z]^\top$ and $\boldsymbol{v} = [v_x,v_y,v_z]^\top$ denote the position and velocity vectors of the center of mass of the spacecraft relative to the origin of the heliocentric frame of reference. The dynamics can be written as:

$$
\begin{align}
\dot{\boldsymbol{r}} &= \boldsymbol{v} = \bm{f}_{\bm{r}}, \nonumber \\
\dot{\boldsymbol{v}} &= -\dfrac{\mu}{r^3} \boldsymbol{r} + \dfrac{T_{\max}}{m} \hat{\boldsymbol{\alpha}}  \delta = \bm{f}_{\bm{v}}, \\
\dot{m} &= - \dfrac{T_{\max}}{c} \delta = f_m
\end{align}
$$

### Explanation of the Dynamics:

1. **Position Update:**

   $$
   \dot{\boldsymbol{r}} = \boldsymbol{v}
   $$

   The velocity vector $\boldsymbol{v}$ represents the rate of change of position $\boldsymbol{r}$.

2. **Velocity Update (Equation of Motion):**

   $$
   \dot{\boldsymbol{v}} = -\dfrac{\mu}{r^3} \boldsymbol{r} + \dfrac{T_{\max}}{m} \hat{\boldsymbol{\alpha}}  \delta
   $$

   - The term $-\dfrac{\mu}{r^3} \boldsymbol{r}$ accounts for the gravitational acceleration due to the central body.
   - The term $\dfrac{T_{\max}}{m} \hat{\boldsymbol{\alpha}}  \delta$ represents the acceleration from an external thrust force, where $\boldsymbol{\alpha}$ is the direction of the thrust and $\delta$ is the amount of propulsion.

3. **Mass Change Due to Propellant Use:**

   $$
   \dot{m} = - \dfrac{T_{\max}}{c} \delta
   $$

   The spacecraft loses mass over time due to fuel consumption, where $c$ is the exhaust velocity of the propellant.

These equations describe the motion of a spacecraft under the influence of a single gravitational body and an external thrust force.



In [ ]:
import sympy as sp
import numpy as np

mu = sp.Symbol('mu')
T_max = sp.Symbol('T_{max}')
c = sp.Symbol('c')

r0,r1,r2 = sp.symbols('r0:3')
v0,v1,v2 = sp.symbols('v0:3')
m = sp.Symbol('m')

r = sp.Matrix(3,1,[r0,r1,r2])
v = sp.Matrix(3,1,[v0,v1,v2])

states = sp.Matrix([r,v,m] )

len_r = sp.sqrt((r.T@r)[0])

alpha = sp.Matrix(3, 1, sp.symbols('alpha0:3'))
delta = sp.Symbol('delta')

state_dot = sp.Matrix([
v,
-mu/(len_r**3)*r + delta*T_max/m*alpha,
-T_max/c*delta,
])

print('Dynamics:')
state_dot

Dynamics:


Matrix([
[                                                           v0],
[                                                           v1],
[                                                           v2],
[T_{max}*alpha0*delta/m - mu*r0/(r0**2 + r1**2 + r2**2)**(3/2)],
[T_{max}*alpha1*delta/m - mu*r1/(r0**2 + r1**2 + r2**2)**(3/2)],
[T_{max}*alpha2*delta/m - mu*r2/(r0**2 + r1**2 + r2**2)**(3/2)],
[                                             -T_{max}*delta/c]])

# Mission

In this example, we want to start the mission at a certain time from Earth and end at a certain time on Mars. Therefore, we should control the spacecraft so that it fulfills our boundary conditions. I will talk about how later, but for now, let's just set up our variables:

$$
\begin{aligned}
&\bm{r}(t_0)=  \bm{r}_0,\\
&\bm{v}(t_0) = \bm{v}_0 ,\\
& m(t_0) = m_0,\\
&\bm{r}(t_f) - \bm{r}_f = \bm{0},\\
&\bm{v}(t_f) - \bm{v}_f = \bm{0}
\end{aligned}
$$

The discrepancy between the initial and final time conditions is deliberate, and we'll get to it later. (Hint: error function!)

Another important aspect of these missions is normalization. Here, we literally deal with astronomical units, and different variables do not necessarily have the same scale. To balance this, we also normalize our parameters.

In [ ]:
AU = 1.496e+8 # km
AUm = AU*1e3 # m
TU = 31536000/(2*np.pi) #s
mu = 132712440018 / (AU**3)*(TU**2) # gravitational parameter
M = 1000 #kg

m_0 = 1000/M
g0 = 9.8065 # m/s^2
I_sp = 2000 

T_max = 0.5 / AUm * (TU**2) / M
c = I_sp*g0 /AUm * TU

r_0 = np.array([-140699693, -51614428, 980])/AU # Earth's position
v_0 = np.array([9.774596, -28.07828 , 4.337725e-4]) /AU * TU # Earth's velocity

r_f = np.array([-172682023, 176959469, 7948912])/AU # Mars' position
v_f = np.array([-16.427384, -14.860506, 9.21486e-2])/AU*TU # Mars' velocity

t_f = 348.795  * 24 * 3600 / TU # time of flight

# Objective

For efficiency, we need to design trajectories that consume as little fuel as possible, so we introduce this objective:

$$J =  \int_{t_0}^{t_f} \dfrac{T_{\max}}{c}\delta(t) \, dt$$

which essentially sums up all the fuel used throughout the trajectory. Using what's called the **Indirect Method**, we can derive a **continuous expression** for the optimal control we seek. Although a full derivation is beyond the scope of this tutorial, I will provide explanations that should be taken with a grain of salt.

To optimize for the objective, we take an approach similar to Lagrange multipliers in multivariable calculus. You might wonder, "but the objective is not constrained." However, it actually is—it must satisfy the previously introduced dynamics. Through the **Euler-Lagrange** equations, we arrive at the following expression, known as the **Hamiltonian**:

$$
H = \dfrac{T_{\max}}{c} \delta + \boldsymbol{\lambda}_{\bm{r}}^\top \bm{f}_{\bm{r}} + \boldsymbol{\lambda}_{\bm{v}}^\top \bm{f}_{\bm{v}} + \lambda_m f_m
$$

Here, instead of Lagrange multipliers, we use costates ($\lambda$), where each $\lambda$ corresponds to a state variable and is multiplied by its associated dynamics. A key difference from the classical Lagrange multipliers is that $\lambda$ values are continuous functions of time and evolve according to the differential equation:

$$\dot{\boldsymbol{\lambda}} = -\left[\dfrac{\partial H}{\partial \boldsymbol{x}}\right]^\top$$

where $\boldsymbol{x}$ represents the state variables (position, velocity, and mass). To find the optimal control, we minimize the Hamiltonian. It turns out that to control the thrust direction ($\boldsymbol{\alpha}$), we must align it in the opposite direction of the costate corresponding to velocity ($\boldsymbol{\lambda}_{\boldsymbol{v}}$). Since direction is a unit vector, we normalize it:

$$\hat{\boldsymbol{\alpha}}^*  = -\frac{\boldsymbol{\lambda_{\boldsymbol{v}}}}{\|\boldsymbol{\lambda_{\boldsymbol{v}}}\|}$$

As for the thrust magnitude ($\delta$), the Hamiltonian is a linear function of it, meaning it could theoretically push the Hamiltonian toward $-\infty$. However, since $\delta$ is constrained to $[0,1]$, we use **Pontryagin's Minimum Principle** to determine the optimal switching behavior. This principle states that the control should be chosen to minimize the Hamiltonian at every moment while respecting constraints on the control variable.

Therefore, the thrust switches between its two extreme values, 0 and 1, meaning the engine is either fully on or completely off during the mission. This decision is governed by the **switching function** $S$, leading to the following expression for thrust:

$$
\delta^* \begin{cases}
    =  1      & \text{if } S > 0, \\
    \in   [0,1] & \text{if } S = 0, \\
    =  0      & \text{if } S \leq 0,
\end{cases}  \quad \text{with} \quad  S = \frac{c\|\bm{\lambda}_{\bm{v}}\|}{m} + \lambda_m - 1,
$$


In [13]:
costates = sp.Matrix(7, 1, sp.symbols('lambda0:7'))

# define Hamiltonian
H = T_max/c*delta  + (costates.T@ state_dot)[0]

# define costate ODE
costate_dot = -H.diff(states)

# Extract lambda_v
lambda_v = costates[3:6,:]
lambda_v_len = sp.sqrt((lambda_v.T@lambda_v)[0])
# Optimal direction
alpha_op = -lambda_v/lambda_v_len
# Define Switching Function
switch = lambda_v_len*c/m + costates[-1] - 1        

# Combine both states and costates into a compact vector
Z = sp.Matrix([states,costates])
Zdot = sp.Matrix([state_dot,costate_dot])

print('State & Costate Dynamics:')
Zdot

State & Costate Dynamics:


Matrix([
[                                                                                                                                                                                              v0],
[                                                                                                                                                                                              v1],
[                                                                                                                                                                                              v2],
[                                                                                                                                   T_{max}*alpha0*delta/m - mu*r0/(r0**2 + r1**2 + r2**2)**(3/2)],
[                                                                                                                                   T_{max}*alpha1*delta/m - mu*r1/(r0**2 + r1**2 + r2**2)**(3/2)],
[          

# Regularization

You might have noticed that we are not quite finished with our optimal control in the code as there is no indication of optimal thrust amount and we haven't substitued the optmal control expressions into our dynamics yet. This is because due to its switching behavior, it is hard to propagte trajectories and even more importantly as we will see in a bit, we need to solve highly nonlinear equations to satisfy boundary conditions. With a control that switches between two extremes, solving the boundary conditions is slow and unstable. Therefore, instead of using the step function for thrust amount, we resort to functions with similar behavior. The most popular choice for such functions is $\tanh$. However, in this tutorial we use the following regularization function and call it $L2$:

$$
\delta^*_\text{L2} \approx \delta_\text{L2} (S;\rho) = 0.5 \left[1 + \frac{S}{\sqrt{S^2 + \rho^2}} \right]
$$

Note that the two functions behave very similar for our purpose. As can be seen from the animation below

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from IPython.display import Video

# Increase font sizes and overall quality
plt.rcParams.update({
    "font.size": 14, 
    "axes.titlesize": 16,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
})

# Define the function δ*_L2
def delta_L2(S, rho):
    return 0.5 * (1 + S / np.sqrt(S**2 + rho**2))

# Create S values for plotting
S = np.linspace(-5, 5, 500)

# Number of frames and sequence of rho values
num_frames = 200
rho_values = np.linspace(1, 1e-5, num_frames)

# Set up the figure with higher DPI for better resolution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), dpi=200)

# ---------------------
# Left Panel: Function Plot
# ---------------------
ax1.set_title(r"Plot of $\delta^*_{L2}(S;\rho)$")
ax1.set_xlabel("S")
ax1.set_ylabel(r"$\delta^*_{L2}$")
ax1.set_xlim(-5, 5)
ax1.set_ylim(0, 1.05)
(line,) = ax1.plot([], [], lw=2, color='blue')
# Annotation for current ρ value
text_rho = ax1.text(0.05, 0.9, '', transform=ax1.transAxes)

# ---------------------
# Right Panel: ρ Timeline
# ---------------------
ax2.set_title(r"$\rho$ Decrease Timeline")
ax2.set_xlabel("Frame")
ax2.set_ylabel(r"$\rho$")
ax2.set_xlim(0, num_frames - 1)
ax2.set_ylim(0, 1)
(ax2_timeline,) = ax2.plot(np.arange(num_frames), rho_values, 'r-', lw=1)
(current_marker,) = ax2.plot([], [], 'bo', markersize=8)

# Initialization function
def init():
    line.set_data([], [])
    text_rho.set_text('')
    current_marker.set_data([], [])
    return line, text_rho, current_marker

# Update function for each frame
def update(frame):
    current_rho = rho_values[frame]
    y = delta_L2(S, current_rho)
    line.set_data(S, y)
    text_rho.set_text(rf"$\rho = {current_rho:.5f}$")
    current_marker.set_data(frame, current_rho)
    return line, text_rho, current_marker

# Create the animation
ani = FuncAnimation(fig, update, frames=num_frames, init_func=init, blit=True, interval=50, repeat=True)

# Close the static figure so it doesn't display in the output
plt.close(fig)

# Save the animation as an MP4 file with high quality settings.
writer = FFMpegWriter(fps=20, metadata=dict(artist='Your Name'), bitrate=3000)
ani.save("high_quality_animation.mp4", writer=writer)

# Embed the video in the notebook.
Video("high_quality_animation.mp4", embed=True)
